In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import h5py
import pandas as pd
import numpy as np

from pathlib import Path
import re

import itertools
sns.set_style('whitegrid')
sns.set_context('talk',rc={'font.family': 'sans-serif', 'font.sans-serif':['Helvetica Neue']})

In [ ]:
simulation_df = []
for filename in Path('../output/julia_summaries/').glob('*'):
    with h5py.File(filename, 'r') as file:
        for run in file.values():
            simulation_df.append(pd.DataFrame(data={'eGFP': run['final_mRNA'][0,:], 'mRuby2': run['final_mRNA'][1,:]}))
            simulation_df[-1]['topo_rate'] = run.attrs['rates.topo']
            simulation_df[-1]['induction'] = run.attrs['gene.base_rate'][0]
            simulation_df[-1]['condition'] = run.attrs['comment'].decode('utf-8')
            simulation_df[-1]['is_plasmid'] = run.attrs['bcs.is_circular']
            simulation_df[-1]['sc_dependent'] = run.attrs['rates.sc_dependent']
simulation_df = pd.concat(simulation_df)

In [ ]:
main_condition_palette = {
    '2_gene.tandem': '#A58245',
    '3_gene.tandem': '#F1BE64',
    '2_gene.divergent': '#18B7CE',
    '3_gene.divergent': '#1DE2FF',
    '2_gene.convergent': '#AC4140',
    '3_gene.convergent': '#F85D5C',
}
light_background_palette = {k:v + '30' for k, v in main_condition_palette.items()}
plot_order = [
    '2_gene.tandem', '3_gene.tandem',
    '2_gene.divergent', '3_gene.divergent',
    '2_gene.convergent', '3_gene.convergent'
]

In [ ]:
plot_df = simulation_df.copy()
plot_df = plot_df[(plot_df['topo_rate'] == min(plot_df['topo_rate'])) & (plot_df['is_plasmid'] == 1) & (plot_df['sc_dependent'] == 1.0)]
plot_df['log mRuby2'] = np.log10(plot_df['mRuby2'] + 1)
ax = sns.kdeplot(x='log mRuby2', hue='condition', palette=main_condition_palette, hue_order=plot_order, bw_adjust=2, fill=True, data=plot_df)
ax.legend_.set_bbox_to_anchor((1, .1))
ax.legend_._set_loc(3)
#sns.legend(loc='upper right', bbox_to_anchor=(1.5,1))
plt.show()

plot_df = simulation_df.copy()
plot_df = plot_df[(plot_df['topo_rate'] == min(plot_df['topo_rate'])) & (plot_df['is_plasmid'] == 0) & (plot_df['sc_dependent'] == 1.0)]
plot_df['log mRuby2'] = np.log10(plot_df['mRuby2'] + 1)
ax = sns.kdeplot(x='log mRuby2', hue='condition', palette=main_condition_palette, hue_order=plot_order, bw_adjust=2, fill=True, data=plot_df)
ax.legend_.set_bbox_to_anchor((1, .1))
ax.legend_._set_loc(3)
plt.show()

In [ ]:
sns.set_style('whitegrid')
for is_plasmid in [0, 1]:
    plot_df = simulation_df.copy()
    plot_df = plot_df[(plot_df['topo_rate'] == min(plot_df['topo_rate'])) & (plot_df['is_plasmid'] == is_plasmid) & (plot_df['induction'] < .008) & (plot_df['sc_dependent'] == 1.0)]
    plot_df['log mRuby2'] = np.log10(plot_df['mRuby2'] + 1)
    plot_df['log eGFP'] = np.log10(plot_df['eGFP'] + 1)

    grid = sns.FacetGrid(plot_df, col='induction', row='condition',
            row_order=[x for x in plot_order if x.startswith('2_gene')],
            margin_titles=True, despine=False)
    grid.map(sns.scatterplot,
        'log eGFP', 'log mRuby2', color='#303030', alpha=0.4, s=10)
    grid.fig.subplots_adjust(wspace=0, hspace=0)
    for axis in grid.axes[0]:
        dox = float(axis.get_title().split('=')[1].strip())
        if dox == 0:
            axis.set_title('Uninduced')
        else:
            axis.set_title('{:.4f}x induction'.format(dox / max(plot_df['induction'])))
    # Adjustment from https://cduvallet.github.io/posts/2018/11/facetgrid-ylabel-access
    for axis in grid.axes.flat:
        if axis.texts:
            txt = axis.texts[0]
            axis.text(*txt.get_unitless_position(),
                    txt.get_text().split('=')[1].strip(),
                    transform=axis.transAxes,
                    va='center')
            axis.texts[0].remove()

    for (row_val,col_val), ax in grid.axes_dict.items():
        ax.set_facecolor(light_background_palette[row_val])
    grid.fig.subplots_adjust(top=0.9)
    grid.fig.suptitle('Simulations with {} boundary conditions'.format('circular' if is_plasmid == 1.0 else 'linear'))
    plt.savefig('../img/t2.0/simulation_plasmid_facets_plasmid-{}.png'.format(is_plasmid), bbox_inches='tight', dpi=300)
    plt.show()

In [ ]:
for sc in [1.0, 0.0]:
    plot_df = simulation_df.copy()
    plot_df = plot_df[(plot_df['topo_rate'] == min(plot_df['topo_rate'])) & (plot_df['is_plasmid'] == 0)  & (plot_df['induction'] < 0.008) & (plot_df['sc_dependent'] == sc)]
    plot_df = plot_df.dropna()
    # Normalize by the max value at induction = 0
    plot_df['mRuby2'] /= np.mean(plot_df[plot_df['induction'] < 0.000084]['mRuby2'])
    plot_df['induction'] /= np.max(plot_df['induction'])
    #plot_df.groupby(['condition', 'induction']).mean().reset_index()
    sns.lineplot(data=plot_df, x='induction', y='mRuby2', hue='condition', ci="sd", linewidth=5, palette=main_condition_palette, hue_order=[x for x in plot_order if x.startswith('2_gene')])
    plt.ylabel('Reporter output')
    plt.xlabel('Induction level') 
    plt.rcParams.update({'font.family': 'Arial', 'font.size': 16})
    plt.legend(['Tandem', 'Divergent', 'Convergent'], loc='upper center')
    plt.gca().grid(False)
    plt.tick_params(axis='both', which='both', bottom=True, top=False, labelbottom=True, right=False, left=True, labelleft=True) 
    plt.xlim([0,1])
    plt.ylim([0,1.4])
    plt.gca().legend_.set_bbox_to_anchor((1.25, .9))
    plt.savefig('../img/t2.0/sim_compare_{}.svg'.format(sc), bbox_inches='tight')
    plt.show()

## Plot flow data mean, STD, CV, and Fano

In [ ]:
plot_df = simulation_df.copy()
plot_df = plot_df[(plot_df['topo_rate'] == min(plot_df['topo_rate'])) & (plot_df['is_plasmid'] == 1)  & (plot_df['induction'] < 0.008) & (plot_df['sc_dependent'] == 1.0)]
plot_df = plot_df.dropna()
# Normalize by the max value at induction = 0
plot_df['mRuby2'] /= np.mean(plot_df[plot_df['induction'] < 0.000084]['mRuby2'])
plot_df['induction'] /= np.max(plot_df['induction'])
mean_plot_data = plot_df.groupby(['condition', 'induction'])['mRuby2'].mean().rename('mean')
std_plot_data = plot_df.groupby(['condition', 'induction'])['mRuby2'].std().rename('std')
cv_plot_data = (std_plot_data / mean_plot_data).rename('cv')
fano_plot_data = (std_plot_data**2 / mean_plot_data).rename('fano')
plot_df = pd.concat([mean_plot_data, std_plot_data, cv_plot_data, fano_plot_data], axis=1).reset_index()
for label in ['mean', 'std', 'cv', 'fano']:
    sns.lineplot(data=plot_df, x='induction', y=label, hue='condition', linewidth=5, palette=main_condition_palette, hue_order=[x for x in plot_order if x.startswith('2_gene')])
    plt.savefig(f'../img/t2.0/simulation_summary_{label}.svg')
    plt.show()

In [ ]:
sns.set_style('whitegrid')
for is_plasmid in [0]:
    plot_df = simulation_df.copy()
    plot_df = plot_df[(plot_df['topo_rate'] == min(plot_df['topo_rate'])) & (plot_df['is_plasmid'] == is_plasmid) & (plot_df['induction'] < .008) & (plot_df['sc_dependent'] == 0.0)]
    plot_df['log mRuby2'] = np.log10(plot_df['mRuby2'] + 1)
    plot_df['log eGFP'] = np.log10(plot_df['eGFP'] + 1)

    grid = sns.FacetGrid(plot_df, col='induction', row='condition',
            row_order=[x for x in plot_order if x.startswith('2_gene')],
            margin_titles=True, despine=False)
    grid.map(sns.scatterplot,
        'log eGFP', 'log mRuby2', color='#303030', alpha=0.4, s=10)
    grid.fig.subplots_adjust(wspace=0, hspace=0)
    for axis in grid.axes[0]:
        dox = float(axis.get_title().split('=')[1].strip())
        if dox == 0:
            axis.set_title('Uninduced')
        else:
            axis.set_title('{:.4f}x induction'.format(dox / max(plot_df['induction'])))
    # Adjustment from https://cduvallet.github.io/posts/2018/11/facetgrid-ylabel-access
    for axis in grid.axes.flat:
        if axis.texts:
            txt = axis.texts[0]
            axis.text(*txt.get_unitless_position(),
                    txt.get_text().split('=')[1].strip(),
                    transform=axis.transAxes,
                    va='center')
            axis.texts[0].remove()

    for (row_val,col_val), ax in grid.axes_dict.items():
        ax.set_facecolor(light_background_palette[row_val])
    grid.fig.subplots_adjust(top=0.9)
    grid.fig.suptitle('Simulations with {} boundary conditions'.format('circular' if is_plasmid == 1.0 else 'linear'))
    plt.savefig('../img/t2.0/sc_independent_plots.png', bbox_inches='tight', dpi=300)
    plt.show()

In [ ]:
sns.set_style('whitegrid')
plot_df = simulation_df.copy()
plot_df = plot_df[(plot_df['condition'] == '2_gene.divergent') & (plot_df['is_plasmid'] == 1.0) & (plot_df['induction'] < .008) & (plot_df['sc_dependent'] == 1.0)]
plot_df['log mRuby2'] = np.log10(plot_df['mRuby2'] + 1)
plot_df['log eGFP'] = np.log10(plot_df['eGFP'] + 1)

grid = sns.FacetGrid(plot_df, col='induction', row='topo_rate',
        margin_titles=True, despine=False)
grid.map(sns.scatterplot,
    'log eGFP', 'log mRuby2', color='#303030', alpha=0.4, s=10)
grid.fig.subplots_adjust(wspace=0, hspace=0)
for axis in grid.axes[0]:
    dox = float(axis.get_title().split('=')[1].strip())
    if dox == 0:
        axis.set_title('Uninduced')
    else:
        axis.set_title('{:.4f}x induction'.format(dox / max(plot_df['induction'])))
# Adjustment from https://cduvallet.github.io/posts/2018/11/facetgrid-ylabel-access
for axis in grid.axes.flat:
    if axis.texts:
        txt = axis.texts[0]
        axis.text(*txt.get_unitless_position(),
                '{:.2E} topo rate'.format(float(txt.get_text().split('=')[1].strip())),
                transform=axis.transAxes,
                va='center')
        axis.texts[0].remove()

for (row_val,col_val), ax in grid.axes_dict.items():
    ax.set_facecolor(light_background_palette['2_gene.divergent'])
grid.fig.subplots_adjust(top=0.85)
grid.fig.suptitle('2-gene divergent, circular BCs')
plt.savefig('../img/t2.0/simulation_2-gene_divergent_plasmid_sweep.png', bbox_inches='tight', dpi=300)
plt.show()

In [ ]:
plot_df.induction.unique()

In [ ]:
sns.set_style('ticks')
sns.set_context('paper')
plt.figure(figsize=(2,1))
plot_df['recentered_EGFP'] = plot_df['log eGFP'] + 0.5
g = sns.jointplot(data=plot_df[(np.abs(plot_df.induction - 1.481e-3) < 1e-4) | (np.abs(plot_df.induction - 6.249e-3) < 1e-4)],
              x='recentered_EGFP', y='log mRuby2', hue='induction', fill=True, kind='kde', alpha=0.5, legend=None,
              palette = {plot_df.induction.unique()[2]:'#555555', plot_df.induction.unique()[3]:'#25ada3'})
g.ax_marg_x.remove()
g.ax_joint.set_xlabel('Sim. log EGFP')
g.ax_joint.set_ylabel('Sim. log mRuby2')
plt.gcf().set_figwidth(2)
plt.gcf().set_figheight(1)
sns.despine()
plt.savefig('../img/simulated_divergent_transfection.svg', bbox_inches='tight')
plt.show()

In [ ]:
sns.set_style('whitegrid')
plot_df = simulation_df.copy()
plot_df = plot_df[(plot_df['condition'] == '2_gene.convergent') & (plot_df['is_plasmid'] == 0.0) & (plot_df['induction'] < .008) & (plot_df['sc_dependent'] == 1.0)]
plot_df['log mRuby2'] = np.log10(plot_df['mRuby2'] + 1)
plot_df['log eGFP'] = np.log10(plot_df['eGFP'] + 1)

grid = sns.FacetGrid(plot_df, col='induction', row='topo_rate',
        margin_titles=True, despine=False)
grid.map(sns.scatterplot,
    'log eGFP', 'log mRuby2', color='#303030', alpha=0.4, s=10)
grid.fig.subplots_adjust(wspace=0, hspace=0)
for axis in grid.axes[0]:
    dox = float(axis.get_title().split('=')[1].strip())
    if dox == 0:
        axis.set_title('Uninduced')
    else:
        axis.set_title('{:.4f}x induction'.format(dox / max(plot_df['induction'])))
# Adjustment from https://cduvallet.github.io/posts/2018/11/facetgrid-ylabel-access
for axis in grid.axes.flat:
    if axis.texts:
        txt = axis.texts[0]
        axis.text(*txt.get_unitless_position(),
                '{:.2E} topo rate'.format(float(txt.get_text().split('=')[1].strip())),
                transform=axis.transAxes,
                va='center')
        axis.texts[0].remove()

for (row_val,col_val), ax in grid.axes_dict.items():
    ax.set_facecolor(light_background_palette['2_gene.divergent'])
grid.fig.subplots_adjust(top=0.85)
grid.fig.suptitle('2-gene convergent, linear BCs')
plt.savefig('../img/t2.0/simulation_2-gene_convergent_linear_sweep.png', bbox_inches='tight', dpi=300)
plt.show()